In [1]:
import glob
import gradio as gr
from Proxy import Proxy
from collections import deque

import html
import os
import re
import time
from pathlib import Path

import markdown
from PIL import Image, ImageOps
from ContextEntry import ContextEntry


def getProxyList():
  return Proxy.GetProxyList()

items = getProxyList()

activeProxy = Proxy("Husserl")

#CSS loader
cwd = os.getcwd()
workPath = os.path.join(cwd, 'css')
file_list = glob.glob(os.path.join(workPath, 'chatSylte.css'))
filename = file_list[0]
with open(filename) as f:
    chatStyle = f.read()
    

def replace_blockquote(m):
    return m.group().replace('\n', '\n> ').replace('\\begin{blockquote}', '').replace('\\end{blockquote}', '')
    
    
def convert_to_markdown(string):
  
    # Blockquote
    string = re.sub(r'(^|[\n])&gt;', r'\1>', string)
    pattern = re.compile(r'\\begin{blockquote}(.*?)\\end{blockquote}', re.DOTALL)
    string = pattern.sub(replace_blockquote, string)

    # Code
    string = string.replace('\\begin{code}', '```')
    string = string.replace('\\end{code}', '```')
    string = re.sub(r"(.)```", r"\1\n```", string)

    result = ''
    is_code = False
    for line in string.split('\n'):
        if line.lstrip(' ').startswith('```'):
            is_code = not is_code

        result += line
        if is_code or line.startswith('|'):  # Don't add an extra \n for tables or code
            result += '\n'
        else:
            result += '\n\n'

    result = result.strip()
    if is_code:
        result += '\n```'  # Unfinished code block

    # Unfinished list, like "\n1.". A |delete| string is added and then
    # removed to force a <ol> or <ul> to be generated instead of a <p>.
    if re.search(r'(\n\d+\.?|\n\*\s*)$', result):
        delete_str = '|delete|'

        if re.search(r'(\d+\.?)$', result) and not result.endswith('.'):
            result += '.'

        result = re.sub(r'(\n\d+\.?|\n\*\s*)$', r'\g<1> ' + delete_str, result)

        html_output = markdown.markdown(result, extensions=['fenced_code', 'tables'])
        pos = html_output.rfind(delete_str)
        if pos > -1:
            html_output = html_output[:pos] + html_output[pos + len(delete_str):]
    else:
        html_output = markdown.markdown(result, extensions=['fenced_code', 'tables'])

    # Unescape code blocks
    pattern = re.compile(r'<code[^>]*>(.*?)</code>', re.DOTALL)
    html_output = pattern.sub(lambda x: html.unescape(x.group()), html_output)

    return html_output    
    
def generateMessageHTML(message: ContextEntry):
  role = message.role
  content = message.content
  roleName = message.roleName
  
  imagePath= os.path.join(workPath, "assets", f"{roleName}.png")
  
  imagePath = glob.glob(imagePath)
  if(os.path.isfile(imagePath[0])):
    imagePath = f"<img src='file/{imagePath[0]}' alt='{roleName}'>"
  else:
    imagePath = ""
  
  if(content):
    if(role == "user"):
      template = f'''
                      <div class="message">
                      <div class="circle-you">
                        {imagePath}
                      </div>
                      <div class="text">
                        <div class="username">
                          {roleName}
                        </div>
                        <div class="message-body">
                          {convert_to_markdown(content)}
                        </div>
                      </div>
                    </div>
      '''
    else:
      template = f'''
                    <div class="message">
                    <div class="circle-bot">
                      {imagePath}
                    </div>
                    <div class="text">
                      <div class="username">
                        {roleName}
                      </div>
                      <div class="message-body">
                        {convert_to_markdown(content)}
                      </div>
                    </div>
                  </div>
      '''
    return template
  else:
    return None
  
  

def generateHTML():
  html = f'<style>{chatStyle}</style><div class="chat" id="chat"><div class="messages">'
  
  history = deque(activeProxy.context.messageHistory)
   
  while len(history) > 0:
    message = history.popleft()
    messageHtml = generateMessageHTML(message)
    if(messageHtml):
      html += messageHtml
  
  html += '''</div></div>  '''
  
  
  return html

def respond(message):
  if(message):
    activeProxy.ReceiveMessage(message=message) 
  html = generateHTML()
  
  return "", html

   
scrollFn = '''
  document.getElementById('chat').scrollTop = document.getElementById('chat').scrollHeight;
  console.log("scrolled");
'''
    
with gr.Blocks() as chat:
  gr.Markdown("# PsychoStasis")
  with gr.Row():
    with gr.Column():
      with gr.Row():
        with gr.Column(scale=4, variant='panel'):
          mainChat = gr.HTML(value = generateHTML(), elem_id="chat_container")
        with gr.Column(scale=1):
          placeholder = gr.HTML(value="")          
          
      txtPrompt = gr.Textbox(label="Prompt", elem_id="prompt")
  with gr.Row():  
    txtPrompt.submit(respond, inputs=[txtPrompt], outputs=[txtPrompt, mainChat]).then(
        lambda: None, None, None, js=f'() => {{{scrollFn}}}')
  
chat.launch(allowed_paths=["."])

loading model Embeddings.Embeddings
model configured
loading model Summarizer.Summarizer
model configured
loading model NER.NER
model configured
loading model SentimentNuanced.Sentiment
model configured
loading model SentimentDiscreet.Sentiment
model configured
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
